<b>pivot_table</b>  
pd.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All'  
简单来说，就是对data按照index选取的列进行分组，然后使用aggfunc指定的方法（默认为mean）对values指定的的列进行聚合。

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('sales-funnel.xlsx')
data.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [27]:
data['Status'] = data['Status'].astype('category').head()

In [ ]:
df["Status"] = df["Status"].astype("category")
df["Status"].cat.set_categories(["won","pending","presented","declined"],inplace=True)

<b>指定index</b>

现在想要计算不同Name的Account、Price、Quantity均值，可以设定index=['Name']，即根据Name进行分组，然后对Account、Price、Quantity求均值。  
PS：不指定values会默认对所有能使用aggfunc的列进行操作。

In [3]:
pd.pivot_table(data,index=['Name'])

,Account,Price,Quantity
Name,,,
Barton LLC,740150.0,35000.0,1.000000
"Fritsch, Russel and Anderson",737550.0,35000.0,1.000000
Herman LLC,141962.0,65000.0,2.000000
Jerde-Hilpert,412290.0,5000.0,2.000000
"Kassulke, Ondricka and Metz",307599.0,7000.0,3.000000
Keeling LLC,688981.0,100000.0,5.000000
Kiehn-Spinka,146832.0,65000.0,2.000000
Koepp Ltd,729833.0,35000.0,2.000000
Kulas Inc,218895.0,25000.0,1.500000


<b>指定values</b>

现在想要得到不同Manager下的不同Rep的Price的均值。

In [4]:
pd.pivot_table(data,index=['Manager','Rep'], values=['Price'])

Price
Manager       Rep                 
Debra Henley  Craig Booker   20000
              Daniel Hilton  38333
              John Smith     20000
Fred Anderson Cedric Moss    27500
              Wendy Yule     44250

<b>指定aggfun</b>

通过指定aggfun，可以实现对指定列元素进行不同的聚合操作，默认为mean。aggfun可以为str，list和dict

In [5]:
pd.pivot_table(data,index=['Manager','Rep'], values=['Price'], aggfunc='sum')

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

此外，还可以对指定的一列或多列元素使用多个聚合操作。

In [6]:
pd.pivot_table(data,index=['Manager','Rep'], values=['Price','Quantity'], aggfunc=[np.sum,len])

sum            len         
                              Price Quantity Price Quantity
Manager       Rep                                          
Debra Henley  Craig Booker    80000        5     4        4
              Daniel Hilton  115000        5     3        3
              John Smith      40000        3     2        2
Fred Anderson Cedric Moss    110000        5     4        4
              Wendy Yule     177000       12     4        4

In [7]:
pd.pivot_table(data,index=['Manager','Rep'], values=['Price','Quantity'], aggfunc={'Price':np.mean,'Quantity':[len,np.std]})

Price Quantity          
                              mean      len       std
Manager       Rep                                    
Debra Henley  Craig Booker   20000        4  0.500000
              Daniel Hilton  38333        3  0.577350
              John Smith     20000        2  0.707107
Fred Anderson Cedric Moss    27500        4  0.500000
              Wendy Yule     44250        4  1.414214

In [8]:
pd.pivot_table(data,index=['Manager','Rep','Product'], values=['Price'], aggfunc=[np.sum])

sum
                                          Price
Manager       Rep           Product            
Debra Henley  Craig Booker  CPU           65000
                            Maintenance    5000
                            Software      10000
              Daniel Hilton CPU          105000
                            Software      10000
              John Smith    CPU           35000
                            Maintenance    5000
Fred Anderson Cedric Moss   CPU           95000
                            Maintenance    5000
                            Software      10000
              Wendy Yule    CPU          165000
                            Maintenance    7000
                            Monitor        5000

<b>指定columns</b>

columns参数是可选的，指定columns可以分割values的结果，下面的这个结果其实跟上面指定index=['Manager','Rep','Product']结果是一致的，不同的是下面的结果中包含NaN值。

In [9]:
pd.pivot_table(data,index=['Manager','Rep'], values=['Price'], columns=['Product'], aggfunc=[np.sum])

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

虽然指定index=['Manager','Rep','Product']得到的结果和指定index=['Manager','Rep']且指定columns=['Product']结果一致，但是我更喜欢用第一种方式。

<b>指定fill_value</b>

针对NaN值，可以通过指定fill_value=0来将NaN值变为0。

In [10]:
pd.pivot_table(data,index=['Manager','Rep'], values=['Price'], columns=['Product'], aggfunc=[np.sum],fill_value=0)

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

<b>指定margins</b>

如果想查看数据总和，可以指定margins=True。

In [11]:
pd.pivot_table(data,index=['Manager','Rep','Product'], values=['Price'], aggfunc=[np.sum, np.mean],margins=True)

sum          mean
                                            Price         Price
Manager       Rep           Product                            
Debra Henley  Craig Booker  CPU           65000.0  32500.000000
                            Maintenance    5000.0   5000.000000
                            Software      10000.0  10000.000000
              Daniel Hilton CPU          105000.0  52500.000000
                            Software      10000.0  10000.000000
              John Smith    CPU           35000.0  35000.000000
                            Maintenance    5000.0   5000.000000
Fred Anderson Cedric Moss   CPU           95000.0  47500.000000
                            Maintenance    5000.0   5000.000000
                            Software      10000.0  10000.000000
              Wendy Yule    CPU          165000.0  82500.000000
                            Maintenance    7000.0   7000.000000
                            Monitor        5000.0   5000.000000
All                                      522000.0  30705.882353

<b>query</b>

使用pivo_table得到结果后，可以使用query方法来进行筛选指定结果。

In [12]:
table = pd.pivot_table(data,index=['Manager','Rep','Product'], values=['Price'], aggfunc=[np.sum, np.mean],margins=True)
table

sum          mean
                                            Price         Price
Manager       Rep           Product                            
Debra Henley  Craig Booker  CPU           65000.0  32500.000000
                            Maintenance    5000.0   5000.000000
                            Software      10000.0  10000.000000
              Daniel Hilton CPU          105000.0  52500.000000
                            Software      10000.0  10000.000000
              John Smith    CPU           35000.0  35000.000000
                            Maintenance    5000.0   5000.000000
Fred Anderson Cedric Moss   CPU           95000.0  47500.000000
                            Maintenance    5000.0   5000.000000
                            Software      10000.0  10000.000000
              Wendy Yule    CPU          165000.0  82500.000000
                            Maintenance    7000.0   7000.000000
                            Monitor        5000.0   5000.000000
All                                      522000.0  30705.882353

In [13]:
table.query('Manager=="Fred Anderson"')

sum     mean
                                          Price    Price
Manager       Rep         Product                       
Fred Anderson Cedric Moss CPU           95000.0  47500.0
                          Maintenance    5000.0   5000.0
                          Software      10000.0  10000.0
              Wendy Yule  CPU          165000.0  82500.0
                          Maintenance    7000.0   7000.0
                          Monitor        5000.0   5000.0

In [14]:
table.query('Manager=="Debra Henley"').query('Rep==["Craig Booker","John Smith"]')

sum     mean
                                         Price    Price
Manager      Rep          Product                      
Debra Henley Craig Booker CPU          65000.0  32500.0
                          Maintenance   5000.0   5000.0
                          Software     10000.0  10000.0
             John Smith   CPU          35000.0  35000.0
                          Maintenance   5000.0   5000.0

如果删除了索引的名称，需要以ilevel_0表示一级索引，ilevel_1表示二级索引，ilevel_2表示三级索引。

In [15]:
table2 = table.copy()
table2.index.names=[None,None,None]

In [16]:
table2.query('ilevel_0=="Fred Anderson"')

sum     mean
                                          Price    Price
Fred Anderson Cedric Moss CPU           95000.0  47500.0
                          Maintenance    5000.0   5000.0
                          Software      10000.0  10000.0
              Wendy Yule  CPU          165000.0  82500.0
                          Maintenance    7000.0   7000.0
                          Monitor        5000.0   5000.0

In [17]:
table2.query('ilevel_0=="Fred Anderson"').query('ilevel_2=="CPU"')

sum     mean
                                  Price    Price
Fred Anderson Cedric Moss CPU   95000.0  47500.0
              Wendy Yule  CPU  165000.0  82500.0

<b>swaplevel和sortlevel</b>

In [18]:
table3 = pd.pivot_table(data,index=['Manager','Rep'], values=['Price'], aggfunc='sum')
table3

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [19]:
table3.swaplevel('Rep','Manager')

,,Price
Rep,Manager,
Craig Booker,Debra Henley,80000
Daniel Hilton,Debra Henley,115000
John Smith,Debra Henley,40000
Cedric Moss,Fred Anderson,110000
Wendy Yule,Fred Anderson,177000


In [20]:
table3.swaplevel(0,1)

,,Price
Rep,Manager,
Craig Booker,Debra Henley,80000
Daniel Hilton,Debra Henley,115000
John Smith,Debra Henley,40000
Cedric Moss,Fred Anderson,110000
Wendy Yule,Fred Anderson,177000


In [21]:
table3.sortlevel(level=1,axis=0,ascending=False)

Price
Manager       Rep                  
Fred Anderson Wendy Yule     177000
Debra Henley  John Smith      40000
              Daniel Hilton  115000
              Craig Booker    80000
Fred Anderson Cedric Moss    110000